Finding dark, non-human, sequences in human samples

In [1]:
cd ~/s3/herv/20180201.MexicanLeukemiaTest/
base=24_S10_L001
ls -l ${base}*z

-r--r--r-- 1 jakewendt staff 596573517 Feb  1 11:17 24_S10_L001_R1_001.fastq.gz
-r--r--r-- 1 jakewendt staff 638346591 Feb  1 11:18 24_S10_L001_R2_001.fastq.gz


In [2]:
export BOWTIE2_INDEXES=~/BOWTIE2_INDEXES

In [3]:
time bowtie2 --threads 8 --very-sensitive -x hg38 \
    -1 ${base}_R1_001.fastq.gz \
    -2 ${base}_R2_001.fastq.gz \
    -S ${base}.sam

11528749 reads; of these:
  11528749 (100.00%) were paired; of these:
    1277974 (11.09%) aligned concordantly 0 times
    6042072 (52.41%) aligned concordantly exactly 1 time
    4208703 (36.51%) aligned concordantly >1 times
    ----
    1277974 pairs aligned concordantly 0 times; of these:
      107633 (8.42%) aligned discordantly 1 time
    ----
    1170341 pairs aligned 0 times concordantly or discordantly; of these:
      2340682 mates make up the pairs; of these:
        1579528 (67.48%) aligned 0 times
        480697 (20.54%) aligned exactly 1 time
        280457 (11.98%) aligned >1 times
93.15% overall alignment rate

real	23m1.887s
user	173m7.075s
sys	0m50.856s


In [5]:
time samtools sort --threads 7 -n -o ${base}.bam ${base}.sam

[bam_sort_core] merging from 14 files...

real	2m6.414s
user	10m5.843s
sys	0m14.409s


In [6]:
time samtools fasta -F 2 --threads 7 -N ${base}.bam > ${base}.nonhg38.fasta

[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 2555948 reads

real	0m7.634s
user	0m22.876s
sys	0m3.544s


In [7]:
export BLASTDB=~/BLAST_DBS

In [8]:
time blastn -query ${base}.nonhg38.fasta -db viral -num_threads 8 2> /dev/null > ${base}.nonhg38.blastn.txt


real	6m5.333s
user	7m33.020s
sys	0m30.436s


In [9]:
time diamond blastx --outfmt 0 --threads 8  --db ~/DIAMOND/viral --query ${base}.nonhg38.fasta 2> /dev/null > ${base}.nonhg38.diamond.txt


real	1m58.435s
user	4m38.562s
sys	1m33.158s


In [10]:
time tblastx -query ${base}.nonhg38.fasta -db viral -num_threads 8 2> /dev/null > ${base}.nonhg38.tblastx.txt



real	159m35.032s
user	1078m1.815s
sys	3m53.238s


OK. So tblastx was taking forever. After a couple hours, still only 5% done. Just don't ever use it.
Let's count the lines in the output files.

In [11]:
grep -c "^Query= " 24_S10_L001.nonhg38.*t

24_S10_L001.nonhg38.blastn.txt:2555948
24_S10_L001.nonhg38.diamond.txt:2555948
24_S10_L001.nonhg38.tblastx.txt:148176


And let's count the reads in the raw fastq files.

In [12]:
zcat 24_S10_L001_R2_001.fastq.gz | grep -c "^@NS"

11528749


And the queried fasta file.

In [13]:
grep -c "^>" 24_S10_L001.nonhg38.fasta

2555948


Good. The reads in the fasta file and the number of queries are the same.

In [14]:
ls -l ${base}.nonhg38.blastn.txt ${base}.nonhg38.diamond.txt

-rw-r--r-- 1 jakewendt staff 607835828 Feb 12 11:31 24_S10_L001.nonhg38.blastn.txt
-rw-r--r-- 1 jakewendt staff 232184755 Feb 12 11:34 24_S10_L001.nonhg38.diamond.txt


Diamond was faster, but the output is smaller.

In [15]:
grep -c "No hits found" ${base}.nonhg38.blastn.txt ${base}.nonhg38.diamond.txt

24_S10_L001.nonhg38.blastn.txt:2550436
24_S10_L001.nonhg38.diamond.txt:2551993


And it found less. Granted both didn't find much.

In [16]:
grep -c "No hits found" ${base}.nonhg38.tblastx.txt

36163


So tblastx found something for about 2/3 of the queries, whilst blastn and diamond only matched 1-2%.